# Experiments

In [ ]:
import os
import IPython.display as ipd

for filename in os.listdir('./test'):
    ipd.Audio('./test/' + filename)

In [ ]:
import os
import pandas as pd
import librosa.display
import matplotlib.pyplot as plt
import tensorflow as tf

for filename in os.listdir('./test'):
    data, sampling_rate = librosa.load('./test/' + filename)

    plt.figure(figsize=(12, 4))
    librosa.display.waveplot(data, sr=sampling_rate)

In [ ]:
import speech_recognition as sr

In [ ]:
r = sr.Recognizer()

In [ ]:
zero = sr.AudioFile('0_jackson_2.wav')

In [ ]:
with zero as source:
    audio = r.record(source)

In [ ]:
print(r.recognize_google(audio))

# This is the main training part:

### training with audio files

In [15]:
%load_ext autoreload
%autoreload 2

from preprocess import *
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.utils import to_categorical

# Second dimension of the feature is dim2
feature_dim_2 = 11

# Save data to array file first
save_data_to_array(max_len=feature_dim_2)

# # Loading train set and test set
X_train, X_test, y_train, y_test = get_train_test()

# # Feature dimension
feature_dim_1 = 20
channel = 1
epochs = 50
batch_size = 100
verbose = 1
num_classes = 3

# Reshaping to perform 2D convolution
X_train = X_train.reshape(X_train.shape[0], feature_dim_1, feature_dim_2, channel)
X_test = X_test.reshape(X_test.shape[0], feature_dim_1, feature_dim_2, channel)

y_train_hot = to_categorical(y_train)
y_test_hot = to_categorical(y_test)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Saving vectors of label - '3': 100%|███████████████████████████████████████████████████| 50/50 [00:00<00:00, 86.71it/s]


### model & prediction

In [16]:
def get_model():
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(2, 2), activation='relu', input_shape=(feature_dim_1, feature_dim_2, channel)))
    model.add(Conv2D(48, kernel_size=(2, 2), activation='relu'))
    model.add(Conv2D(120, kernel_size=(2, 2), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.25))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.4))
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer=keras.optimizers.Adadelta(),
                  metrics=['accuracy'])
    return model

# Predicts one sample
def predict(filepath, model):
    sample = wav2mfcc(filepath)
    sample_reshaped = sample.reshape(1, feature_dim_1, feature_dim_2, channel)
    return get_labels()[0][
            np.argmax(model.predict(sample_reshaped))
    ]

In [17]:
model = get_model()
model.fit(X_train, y_train_hot, batch_size=batch_size, epochs=epochs, verbose=verbose, validation_data=(X_test, y_test_hot))

Train on 88 samples, validate on 59 samples
Epoch 1/100
88/88 [==============================] - ETA: 3s - loss: 4.8838 - acc: 0.333 - ETA: 1s - loss: 5.5576 - acc: 0.266 - ETA: 0s - loss: 6.7225 - acc: 0.266 - ETA: 0s - loss: 7.0968 - acc: 0.266 - ETA: 0s - loss: 7.5254 - acc: 0.253 - 1s 15ms/step - loss: 7.3909 - acc: 0.2614 - val_loss: 1.6124 - val_acc: 0.4068
Epoch 2/100
88/88 [==============================] - ETA: 0s - loss: 3.2988 - acc: 0.400 - ETA: 0s - loss: 3.8586 - acc: 0.400 - ETA: 0s - loss: 3.0384 - acc: 0.450 - ETA: 0s - loss: 2.7663 - acc: 0.440 - 0s 4ms/step - loss: 2.8242 - acc: 0.3977 - val_loss: 1.1599 - val_acc: 0.2881
Epoch 3/100
88/88 [==============================] - ETA: 0s - loss: 1.7212 - acc: 0.400 - ETA: 0s - loss: 1.5430 - acc: 0.433 - ETA: 0s - loss: 1.4911 - acc: 0.377 - ETA: 0s - loss: 1.7612 - acc: 0.350 - ETA: 0s - loss: 1.7412 - acc: 0.346 - 0s 4ms/step - loss: 1.7164 - acc: 0.3409 - val_loss: 1.1323 - val_acc: 0.3390
Epoch 4/100
88/88 [===========

88/88 [==============================] - ETA: 0s - loss: 0.7077 - acc: 0.733 - ETA: 0s - loss: 0.6791 - acc: 0.777 - ETA: 0s - loss: 0.7505 - acc: 0.716 - ETA: 0s - loss: 0.7448 - acc: 0.706 - 0s 4ms/step - loss: 0.7406 - acc: 0.7045 - val_loss: 1.3224 - val_acc: 0.3729
Epoch 32/100
88/88 [==============================] - ETA: 0s - loss: 0.6980 - acc: 0.666 - ETA: 0s - loss: 0.6934 - acc: 0.633 - ETA: 0s - loss: 0.6055 - acc: 0.733 - 0s 3ms/step - loss: 0.5868 - acc: 0.7727 - val_loss: 1.4343 - val_acc: 0.3729
Epoch 33/100
88/88 [==============================] - ETA: 0s - loss: 0.4958 - acc: 0.866 - ETA: 0s - loss: 0.5178 - acc: 0.800 - ETA: 0s - loss: 0.5224 - acc: 0.800 - ETA: 0s - loss: 0.5286 - acc: 0.800 - 0s 3ms/step - loss: 0.5464 - acc: 0.7727 - val_loss: 1.4926 - val_acc: 0.2881
Epoch 34/100
88/88 [==============================] - ETA: 0s - loss: 0.5689 - acc: 0.800 - ETA: 0s - loss: 0.3946 - acc: 0.888 - ETA: 0s - loss: 0.4856 - acc: 0.826 - 0s 3ms/step - loss: 0.4869 - ac

Epoch 63/100
88/88 [==============================] - ETA: 0s - loss: 0.0108 - acc: 1.000 - ETA: 0s - loss: 0.0055 - acc: 1.000 - ETA: 0s - loss: 0.0081 - acc: 1.000 - 0s 3ms/step - loss: 0.0080 - acc: 1.0000 - val_loss: 3.1387 - val_acc: 0.3390
Epoch 64/100
88/88 [==============================] - ETA: 0s - loss: 0.0190 - acc: 1.000 - ETA: 0s - loss: 0.0120 - acc: 1.000 - ETA: 0s - loss: 0.0090 - acc: 1.000 - 0s 3ms/step - loss: 0.0081 - acc: 1.0000 - val_loss: 3.0666 - val_acc: 0.3220
Epoch 65/100
88/88 [==============================] - ETA: 0s - loss: 0.0098 - acc: 1.000 - ETA: 0s - loss: 0.0266 - acc: 1.000 - ETA: 0s - loss: 0.0286 - acc: 0.986 - 0s 3ms/step - loss: 0.0247 - acc: 0.9886 - val_loss: 3.1564 - val_acc: 0.3390
Epoch 66/100
88/88 [==============================] - ETA: 0s - loss: 0.0109 - acc: 1.000 - ETA: 0s - loss: 0.0195 - acc: 1.000 - ETA: 0s - loss: 0.0143 - acc: 1.000 - 0s 3ms/step - loss: 0.0134 - acc: 1.0000 - val_loss: 3.3608 - val_acc: 0.3220
Epoch 67/100
88/

Epoch 96/100
88/88 [==============================] - ETA: 0s - loss: 2.6654e-04 - acc: 1.000 - ETA: 0s - loss: 0.0013 - acc: 1.0000    - ETA: 0s - loss: 0.0079 - acc: 1.000 - 0s 3ms/step - loss: 0.0071 - acc: 1.0000 - val_loss: 4.5608 - val_acc: 0.2712
Epoch 97/100
88/88 [==============================] - ETA: 0s - loss: 0.0129 - acc: 1.000 - ETA: 0s - loss: 0.0046 - acc: 1.000 - ETA: 0s - loss: 0.0034 - acc: 1.000 - 0s 3ms/step - loss: 0.0043 - acc: 1.0000 - val_loss: 4.4696 - val_acc: 0.3390
Epoch 98/100
88/88 [==============================] - ETA: 0s - loss: 0.0068 - acc: 1.000 - ETA: 0s - loss: 0.0054 - acc: 1.000 - ETA: 0s - loss: 0.0049 - acc: 1.000 - 0s 3ms/step - loss: 0.0044 - acc: 1.0000 - val_loss: 4.3428 - val_acc: 0.3051
Epoch 99/100
88/88 [==============================] - ETA: 0s - loss: 0.0340 - acc: 1.000 - ETA: 0s - loss: 0.0177 - acc: 1.000 - ETA: 0s - loss: 0.0092 - acc: 1.000 - 0s 4ms/step - loss: 0.0065 - acc: 1.0000 - val_loss: 4.4416 - val_acc: 0.2881
Epoch 10

### Predict on a new file

In [24]:
print(predict('./luke_02_4.wav', model=model))

1


# Confusion matrix

In [33]:
from sklearn.metrics import classification_report, confusion_matrix

y_pred = model.predict(X_test)

cm = confusion_matrix(y_test, y_pred)

ValueError: Classification metrics can't handle a mix of multiclass and continuous-multioutput targets